In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder


In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()


True     5158
False    4842
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)


In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)


In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])


In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)


Epoch 1/10
209/209 [==============================] - 1s 5ms/step - loss: 1.7819 - mae: 1.0272 - mse: 1.7819 - val_loss: 1.1813 - val_mae: 0.8603 - val_mse: 1.1813
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.4954 - mae: 0.9607 - mse: 1.4954 - val_loss: 1.0762 - val_mae: 0.8229 - val_mse: 1.0762
Epoch 3/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4431 - mae: 0.9411 - mse: 1.4431 - val_loss: 1.0835 - val_mae: 0.8240 - val_mse: 1.0835
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4029 - mae: 0.9250 - mse: 1.4029 - val_loss: 2.5587 - val_mae: 1.3492 - val_mse: 2.5587
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3732 - mae: 0.9170 - mse: 1.3732 - val_loss: 1.1264 - val_mae: 0.8365 - val_mse: 1.1264
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3388 - mae: 0.9061 - mse: 1.3388 - val_loss: 1.1168 - val_mae: 0.8313 - val_mse: 1.1168
Epoch 7/10
209/2

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])


In [12]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()


Predicted val:  7.9173636
Actual val:  7.0988848364

Predicted val:  7.9473414
Actual val:  8.313631900019999

Predicted val:  8.3552265
Actual val:  6.11562314788

Predicted val:  8.622347
Actual val:  7.67429134022

Predicted val:  8.3937645
Actual val:  7.62578964258

Predicted val:  8.375775
Actual val:  7.1870697412

Predicted val:  7.9219112
Actual val:  6.7130758779

Predicted val:  8.368656
Actual val:  7.56846945446

Predicted val:  8.067903
Actual val:  7.43839671988

Predicted val:  7.6035123
Actual val:  6.6248909731



In [13]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [14]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [15]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)


WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…